In [4]:
import tensorflow as tf
import pickle
import os

import numpy as np
import pandas as pd

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from tensorflow.keras.utils import load_img
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'


In [6]:
df = pd.read_json("/Users/liyuxiao/Downloads/nii_project/dataset/1.json")
df["char"] = df["data"].apply(lambda x: x["char"])
df["url"] = df["data"].apply(
    lambda x: "/Users/liyuxiao/Downloads/nii_project/" + x["url"][28:]
)
df_count = (
    df.groupby("char").count().sort_values(by=["data"], ascending=False)[["data"]]
)

single_char = [ele for ele in df_count.index if len(ele) == 1]
df_single_char = df[df["char"].isin(single_char)]
df_single_char.loc[:, "image_file"] = df_single_char["url"].apply(lambda x: x[48:])

In [7]:
path = r"/Users/liyuxiao/Downloads/nii_project/dataset/1"
# change the working directory to the path where the images are located
os.chdir(path)
# this list holds all the image filename
chars = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
    # loops through each file in the directory
    for file in files:
        if (
            file.name.endswith(".jpg")
            and file.name in df_single_char["image_file"].values
        ):
            # adds only the image files to the flowers list
            chars.append(file.name)

In [2]:
# load the model first and pass as an argument
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)


def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224, 224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1, 224, 224, 3)
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True, verbose=0)
    return features


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-04-04 14:38:38.688353: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-04 14:38:38.688666: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
data = {}
p = r"/Users/liyuxiao/Downloads/nii_project/dataset/2/char_features.pkl"

# lop through each image in the dataset
for char in tqdm(chars):
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(
            "/Users/liyuxiao/Downloads/nii_project/dataset/2/" + char, model
        )
        data[char] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p, "wb") as file:
            pickle.dump(data, file)

# get a list of the filenames
filenames = np.array(list(data.keys()))


In [ ]:
with open("/Users/liyuxiao/Downloads/nii_project/dataset/feature_data.pkl", "wb") as fp:
    pickle.dump(data, fp)
    print("dictionary saved successfully to file")